In [1]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import joblib
import warnings
import pandas as pd
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import warnings

In [45]:
def reset():
    global Model_results_df
    Model_results_df = pd.read_csv("Model_results.csv")
    Model_results_df
    
reset()

In [3]:
# Load your dataset
raw_data = pd.read_csv("Final_dataset.csv", parse_dates=["DateTime"], index_col="DateTime")

# Create a date range covering the desired time period
start_date = "2018-09-01"
end_date = "2021-09-30"
freq = "30T"  # 30 minutes frequency
date_range = pd.date_range(start=start_date, end=end_date, freq=freq)

# Create a DataFrame with the date range as index
data = pd.DataFrame(index=date_range)

# Load your dataset and merge with the date range DataFrame
data = data.merge(raw_data, how="left", left_index=True, right_index=True)
data.fillna(0, inplace=True)


data


                     USEP (Milligrams of gold/MWh)  DEMAND (MW)  \
2018-09-01 00:00:00                       0.000000        0.000   
2018-09-01 00:30:00                     446.335250     5516.442   
2018-09-01 01:00:00                     422.275525     5410.561   
2018-09-01 01:30:00                     419.678168     5280.854   
2018-09-01 02:00:00                     441.322807     5197.666   
...                                            ...          ...   
2021-09-29 22:00:00                     708.455791     6512.807   
2021-09-29 22:30:00                     597.321893     6351.307   
2021-09-29 23:00:00                     597.458757     6203.728   
2021-09-29 23:30:00                     597.413136     6052.382   
2021-09-30 00:00:00                     575.239170     5927.635   

                     Total Population (Number)  Holiday  Day  
2018-09-01 00:00:00                        0.0      0.0  0.0  
2018-09-01 00:30:00                  5638676.0      1.0  6.0  
2018-0

In [95]:

# Mute warnings from a specific module
warnings.filterwarnings("ignore", category=ConvergenceWarning, module="statsmodels")


# Create a DataFrame to store MSE values for different alpha values and forecast steps
forcast_steps_to_MSE_df = []

# Iterate through different alpha values
alpha_values = [0.1, 0.2, 0.3, 0.4, 0.5]
for alpha in alpha_values:
    # Fit Exponential Smoothing model
    model = ExponentialSmoothing(data['DEMAND (MW)'], seasonal=None, trend='add', dates=data.index)
    model_fit = model.fit(smoothing_level=alpha)
    
    # Calculate MSE values for different forecast steps
    mse_values = []
    for i in range(1, 5):
        # Make forecasts
        forecast_steps = i
        forecast = model_fit.forecast(steps=forecast_steps)
        
        # Calculate Mean Squared Error
        actual_values = data['DEMAND (MW)'][-forecast_steps:]
        mse = mean_squared_error(actual_values, forecast)
        
        mse_values.append(mse)
    
    forcast_steps_to_MSE_df.append(mse_values)

# Create a DataFrame from the list of lists with MSE values
forcast_steps_to_MSE_df = pd.DataFrame(forcast_steps_to_MSE_df, index=alpha_values, columns=range(1, 5))

# Restore default warning behavior
warnings.resetwarnings()

forcast_steps_to_MSE_df

,1,2,3,4
0.1,428842.965295,357344.339591,288971.340433,232993.209264
0.2,25627.652756,3263.660496,5463.263090,35195.216443
0.3,107.161858,12931.958004,61019.581018,145543.575894
0.4,11625.246236,60350.153219,153573.889850,291033.329335
0.5,24914.388265,89729.269999,202088.168638,361067.948543


,MSE
1,25627.652756
2,3263.660496
3,5463.263090
4,35195.216443


,Model,MSE (Lowest),Lowest MSE step,forcast_steps_to_MSE_df
0,ExponentialSmoothing,,25627,NaN


In [115]:
reset()

In [116]:
# log MSE
# Assign the minimum MSE value to the 'MSE (Lowest)' column
Model_results_df.loc[Model_results_df['Model'] == "ExponentialSmoothing", 'MSE (Lowest)'] = forcast_steps_to_MSE_df.min().min()

# Get the row and column index of the overall minimum value
min_row, min_col = forcast_steps_to_MSE_df.stack().idxmin()

# Assign the alpha value for the lowest MSE to the 'Lowest MSE alpha' column
Model_results_df.loc[Model_results_df['Model'] == "ExponentialSmoothing", 'Lowest MSE step'] = min_col

# Assign the step value for the lowest MSE to the 'Lowest MSE step' column
Model_results_df.loc[Model_results_df['Model'] == "ExponentialSmoothing", 'Lowest MSE alpha'] = min_row




In [118]:
Model_results_df

,Model,MSE (Lowest),Lowest MSE step,Lowest MSE alpha
0,ExponentialSmoothing,107.161858,1.0,0.3


In [119]:
model_fit = model.fit(smoothing_level= Model_results_df.loc[Model_results_df['Model'] == "ExponentialSmoothing", 'Lowest MSE alpha'])

best_step = Model_results_df.loc[Model_results_df['Model'] == "ExponentialSmoothing", 'Lowest MSE step']

C:\Users\thacu\PycharmProjects\Data_sci_pw\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [120]:
# Save the trained model using joblib
model_filename = 'exponential_smoothing_model.pkl'
joblib.dump(model_fit, model_filename)
joblib.dump(best_step, "best_step_"+model_filename )
print("Model saved as", model_filename)

Model saved as exponential_smoothing_model.pkl
